In [2]:
import numpy as np
import pandas as pd

We aim to see how many people say they have a mentall illness by looking at regular expressions in the form of 'I was diagnosed with X'.

In [3]:
df = pd.read_json('reduced_tweets.json', lines=True)

In [107]:
df.head(1)

,author_gender,lang,main,published,sentiment,source_followers,source_following,source_location,source_spam_probability,tags
0,FEMALE,fr,en espérant que 2016 soit meilleur que 2015 quand même,2016-01-01T00:30:04Z,NEUTRAL,80,140,Saint-Maurice,0,NaN


We define the list of desease we want to look at

In [38]:
diseases = ['anxiety','anorexie', 'bipolar', 'borderline', \
            'depression', 'eating disorder', 'ocd',\
            'ptsd', 'schizo', 'seasonal affective']

We define functions that give use people stating they were dignosed as or stating that they have the disease:

In [41]:
def find_diagnosed(df, disease, main):
    return df[df[main].str.contains(disease) & df[main].str.contains('diagnos')]

In [85]:
def find_affected(df, disease, main, lang):
    if(lang == 'fr'):
        personal = 'j '
        suffer = 'je souffre'
    elif(lang == 'de'):
        personal = 'ich habe'
        suffer = 'ich bin'
    elif(lang == 'en'):
        personal = 'i have'
        suffer = 'i suffer'
    condition1 = df[main].str.contains(disease) & df[main].str.contains(personal)
    condition2 = df[main].str.contains(disease) & df[main].str.contains(suffer)
    return df[df.lang == lang][condition1 | condition2]

We try the functions with an example, in this case we use anxiety as an example:

In [73]:
affected = find_affected(df, 'anxi', 'main', 'en')

/Users/lguillain/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [77]:
affected.main.head()

227108    loving that our next comp is the 16 and i have finals the 13-15 and if i don't pass one i don't graduate here come the anxiety attacks️   
376093    sikkwave: *i have anxiety dont make fun of me* sikkwave: *posts selfies of jen, calls her ugly, makes fun of her age & thinks shes badass*
423890    why am i feeling anxious i have literally no reason to but i am thanks                                                                    
518874      i have anxiety                                                                                                                          
652445    i have the worst anxiety tonight...                                                                                                       
Name: main, dtype: object

These results look good. but, there are very few of them:

In [78]:
len(affected)

41

We now look at gender distribution & sentiment:

In [75]:
affected.author_gender.value_counts()

UNKNOWN    21
FEMALE     13
MALE       7 
Name: author_gender, dtype: int64

In [76]:
affected.sentiment.value_counts()

NEGATIVE    28
NEUTRAL     13
Name: sentiment, dtype: int64

We now look at all diseases we listed, which are the most talked about diseases:

In [56]:
frames = []
for disease in diseases:
    frames.append(find_diagnosed(df, disease, 'main'))
diagnosed = pd.concat(frames)

In [105]:
pd.set_option('display.max_colwidth', -1)
print('We find : ', len(diagnosed), 'tweets')
print('We find the following tweets')
print(diagnosed.main.head())

We find :  18 tweets
We find the following tweets
376273     has emerging markets already been diagnosed with bipolar disorder?                                                                       
1197674    et puis c'est le genre de maladies qui se diagnostiquent à l'age adulte comme la bipolarité                                              
1758348    meine selbstdiagnose: borderline basic-bitch                                                                                             
3735199    une certaine personne me diagnostique borderline sur la base d'un test interner et me reproche des choses que je pourrais faire          
1006658    before you diagnose yourself with depression or low self-esteem, first make shure that you are not, in fact, just surrounded by assholes.
Name: main, dtype: object


In [48]:
frames = []
for disease in diseases:
    frames.append(find_affected(df, disease, 'main', 'english'))
affirming = pd.concat(frames)

In [106]:
print('We find : ',len(affirming), 'tweets')
print('We find the following tweets')
print(affirming.main.head())

We find :  58 tweets
We find the following tweets
227108    loving that our next comp is the 16 and i have finals the 13-15 and if i don't pass one i don't graduate here come the anxiety attacks️   
376093    sikkwave: *i have anxiety dont make fun of me* sikkwave: *posts selfies of jen, calls her ugly, makes fun of her age & thinks shes badass*
518874      i have anxiety                                                                                                                          
652445    i have the worst anxiety tonight...                                                                                                       
654634    when people post really dumb shit and have pictures of their offspring, i have mild anxiety attacks that those people procreated.         
Name: main, dtype: object


We see that we can only find very few users stating that they have a mental illness



We should also look at stress, a common sign of mental distress:

In [86]:
stress_en = find_affected(df, 'stress', 'main', 'en')
stress_fr = find_affected(df, 'stress', 'main', 'fr')
stress_de = find_affected(df, 'stress', 'main', 'de')

/Users/lguillain/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [87]:
print(len(stress_en), len(stress_de), len(stress_fr))

46 26 13


In [98]:
mental_health = df[df.main.str.contains('mental health') | df.main.str.contains('mental illness')]

In [99]:
len(mental_health)

295

In [102]:
mental_health.author_gender.value_counts()

UNKNOWN    166
FEMALE     66 
MALE       63 
Name: author_gender, dtype: int64

In [103]:
mental_health.sentiment.value_counts() #signifficantly more negative than sample set

NEGATIVE    185
NEUTRAL     110
Name: sentiment, dtype: int64

We try to find the same thing for german:

In [100]:
geistige_gesundheit = df[df.main.str.contains('geistige gesund') | df.main.str.contains('geistig krank')]

And we look at the tweets containing stress:

In [96]:
stress = df[df.main.str.contains('stress')]

In [104]:
stress.main.head() #5259 tweets, do topic model on this

2744    now that it has survived #32c3 (my notebook stress test), i conclude that i’m happy with my thinkpad x1 carbon 2015 (linux); i recommend it.
2870    j'ai commencé à stresser sur twitch en zombies ! mon twitch : zezettedezombie                                                               
3407    wow par contre la sa deviens assez stressant mon allergie !                                                                                 
4884    "sorry dass ich mich 2 monate nicht gemeldet hab, hatte total umzugsstress." *schreibt dieselbe adresse wie seit 5 jahren in den absender*  
6501    ma mère qui stress la à sortir de mon lit... vas y genre j'ai envie de voir les gens qui ont gâché mon réveillon... lol                     
Name: main, dtype: object

We now look at what makes swiss users stressed by applying a topic model

In [108]:
depressed = df[df.main.str.contains('depress')]

In [111]:
depressed #1653 tweets

,author_gender,lang,main,published,sentiment,source_followers,source_following,source_location,source_spam_probability,tags
9845,MALE,en,how to find something to make that matters? #depressed,2016-02-01T18:41:28Z,NEUTRAL,487,430,Basel,0,[depressed]
14998,UNKNOWN,en,how to save an economy from a great depression,2016-02-01T18:44:42Z,NEUTRAL,2833,887,Switzerland,0,NaN
24725,MALE,de,depressione wege dere lap. weiss gar nöd was lerne. lol. 50 ordner?,2016-03-01T20:47:14Z,NEUTRAL,61,64,Switzerland,0,NaN
24995,FEMALE,fr,fanbase de maniaco-depressifs j'te dis...,2016-03-01T21:55:27Z,NEUTRAL,801,613,Switzerland,0,NaN
31947,UNKNOWN,en,"honestly, if my #moods depended on #twitter follows and unfollows - i'd be swinging perpetually between ecstasy and depression!",2016-04-01T11:07:41Z,NEUTRAL,41,50,Switzerland,0,"[Twitter, moods]"
40344,UNKNOWN,de,git doch nüt schöners als sunntig diheime si und sich langwile und depressione schiebe wegem mäntig,2016-05-01T16:40:42Z,NEUTRAL,61,64,Switzerland,0,NaN
41519,FEMALE,en,"i would visit but hospitals make me very depressed , i am deeply sorry",2016-05-01T08:32:24Z,NEGATIVE,466,2030,Switzerland,0,NaN
48536,UNKNOWN,en,i ended up talking about my depression and the thing that i've been through (which she had no idea of how i was feeling and stuff),2016-06-01T20:26:27Z,NEGATIVE,122,521,Switzerland,0,NaN
51555,UNKNOWN,en,"i'm going to be depressed on sunday evening, lol...",2016-06-01T21:26:27Z,NEGATIVE,198,584,Switzerland,0,NaN
54469,MALE,en,depressing that ramsey will miss his country's most important game in history because of a second yellow #wal,2016-07-01T21:14:51Z,NEGATIVE,663,778,"London, UK",0,[WAL]


In [112]:
depressed.sentiment.value_counts()

NEGATIVE    937
NEUTRAL     716
Name: sentiment, dtype: int64

Look at SAD using seasonal prevelance of of sentiments & hourly presence of sentiments.